In [1]:
### PREPARE DF FOR DARK SKY API ######
import sqlite3
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

import pandas as pd
import seaborn as sns

c.execute('''SELECT * FROM Matches WHERE season = 2011;''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
print(len(df))
df.head()

992


,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [2]:
# Remove unwanted columns #
df.drop(['Div','FTHG','FTAG','FTR','AwayTeam','HomeTeam','Season'],axis = 1, inplace = True)

In [3]:
df.head()

,Match_ID,Date
0,1092,2012-03-31
1,1093,2011-12-11
2,1094,2011-08-13
3,1095,2011-11-27
4,1096,2012-02-18


In [9]:
df.set_index('Match_ID',inplace = True)

In [10]:
weather_df = df

In [11]:
weather_df.head()

,Date
Match_ID,
1092,1333152000
1093,1323561600
1094,1313193600
1095,1322352000
1096,1329523200


In [7]:
#change date to unix date time for use in api call
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = (df['Date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [12]:
weather_df.head()

,Date
Match_ID,
1092,1333152000
1093,1323561600
1094,1313193600
1095,1322352000
1096,1329523200


In [20]:
#
weather_df.sort_values(by = 'Date',inplace = True)
weather_df.head()

,Date
Match_ID,
1133,1310688000
1167,1310688000
1551,1310688000
1678,1310774400
1550,1310774400


In [14]:
#### WEATHER API #####

In [14]:
#imports for api call
from bs4 import BeautifulSoup
import requests
import json
import pprint

In [24]:
#prep for api call test
url = 'https://api.darksky.net/forecast/7bae2b226c09c124a036c0499d4f4456/52.52437,13.41053,1333152000?exclude=minutely,hourly,daily,alerts,flags'
response = requests.get(url)
print(response)
pprint.pprint(response.text)

<Response [200]>
('{"latitude":52.52437,"longitude":13.41053,"timezone":"Europe/Berlin","currently":{"time":1333152000,"summary":"Mostly '
 'Cloudy","icon":"partly-cloudy-night","precipIntensity":0,"precipProbability":0,"temperature":46.41,"apparentTemperature":38.98,"dewPoint":39.53,"humidity":0.77,"windSpeed":19.55,"windGust":35.66,"windBearing":281,"cloudCover":0.75,"uvIndex":0,"visibility":6.216},"offset":2}\n')


In [25]:
#keys from the api call to see what we want
response.json().keys()

dict_keys(['latitude', 'longitude', 'timezone', 'currently', 'offset'])

In [26]:
#call the weather
response.json()['currently']['summary']

'Mostly Cloudy'

In [ ]:
#get weather from list of dates per match id
